In [1]:
import sys
import os

# Añade el directorio padre al sys.path
sys.path.append(os.path.abspath("/mnt/netapp2/Store_uni/home/usc/ie/dcr/software/hk"))

# ROOT!
import os, sys
os.environ["ROOTSYS"]   = "/mnt/netapp2/Store_uni/home/usc/ie/dcr/software/ROOT/ROOT_6.26.14/install"
os.environ["BONSAIDIR"] = "/mnt/netapp2/Store_uni/home/usc/ie/dcr/software/hk/hk-BONSAI"
os.environ["WCSIM_BUILD_DIR"] = "/mnt/netapp2/Store_uni/home/usc/ie/dcr/software/hk/WCSim/install/"
sys.path.append(os.path.join(os.environ["ROOTSYS"], "lib"))
sys.path.append(os.path.join(os.environ["BONSAIDIR"], "lib"))
sys.path.append(os.path.join(os.environ["WCSIM_BUILD_DIR"], "lib"))

import ROOT

import uproot
import glob 
import array
import cppyy

cppyy.add_include_path("/mnt/netapp2/Store_uni/home/usc/ie/dcr/software/hk/WCSim/install/include")
cppyy.load_library("/mnt/netapp2/Store_uni/home/usc/ie/dcr/software/hk/WCSim/install/lib/libWCSimRoot.so")

cppyy.add_include_path("/mnt/netapp2/Store_uni/home/usc/ie/dcr/software/hk/hk-BONSAI/bonsai")
cppyy.load_library("/mnt/netapp2/Store_uni/home/usc/ie/dcr/software/hk/hk-BONSAI/libWCSimBonsai.so")

# import hipy.hipy.pltext  as pltext
# import hipy.hipy.utils   as ut
# import matplotlib.pyplot as plt
# import awkward           as ak
import numpy             as np
# import matplotlib.colors as colors
import pandas            as pd

from tqdm import tqdm

Error in <TExMap::Add>: key 140471386123424 is not unique
Error in <TExMap::Add>: key 140471386702288 is not unique
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<TString*,vector<TString> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<string*,vector<string> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<int*,vector<int> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<char*,string> already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<const char*,string> already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<TString*,vector<TString> > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<const TString*,vector<TString> > already in TClassTable
TCla

Welcome to JupyROOT 6.26/14


Warning in <TClassTable::Add>: class TConditionImp already in TClassTable
Warning in <TClassTable::Add>: class TPosixCondition already in TClassTable
Warning in <TClassTable::Add>: class TMutexImp already in TClassTable
Warning in <TClassTable::Add>: class TPosixMutex already in TClassTable
Warning in <TClassTable::Add>: class TThreadImp already in TClassTable
Warning in <TClassTable::Add>: class TPosixThread already in TClassTable
Warning in <TClassTable::Add>: class TThreadFactory already in TClassTable
Warning in <TClassTable::Add>: class TPosixThreadFactory already in TClassTable
Warning in <TClassTable::Add>: class TAtomicCount already in TClassTable
Warning in <TClassTable::Add>: class TCondition already in TClassTable
Warning in <TClassTable::Add>: class TMutex already in TClassTable
Warning in <TClassTable::Add>: class TRWLock already in TClassTable
Warning in <TClassTable::Add>: class TSemaphore already in TClassTable
Warning in <TClassTable::Add>: class TThread already in TCl

In [2]:
def get_offline_run_files(run, base="/mnt/lustre/scratch/nlsas/home/usc/ie/dcr/hk/raw_data"):
    files = glob.glob(f"{base}/{run}/WCTE_offline_R*.root")
    return files

def get_offline_run_tchain(run, limit=0):
    files = get_offline_run_files(run)
    tt = ROOT.TChain("WCTEReadoutWindows")
    count = 0
    for ff in files:
        tt.Add(ff)
        count += 1
        if limit > 0 and count > limit: break
    return tt

# Function to load geometry mapping from a text file
def get_geo_mapping():
    geo = pd.read_csv("/mnt/netapp2/Store_uni/home/usc/ie/dcr/software/hk/WCSim/install/geofile_NuPRISMBeamTest_16cShort_mPMT.txt", 
                      index_col=False,      # Do not use any column as the index
                      sep='\s+',            # Use whitespace as separator
                      skiprows=5,           # Skip the first 5 header lines
                      names=["id","mpmtid","spmtid",
                             "x","y","z","dx","dy","dz", "cyloc"])  # Explicit column names

    return geo

geo = get_geo_mapping()

def getxyz(geo, mpmtids, posids):
    # Build a single lookup dictionary: {(mpmtid, spmtid): (x, y, z, id)}
    lookup = {
        (row.mpmtid, row.spmtid): (row.x, row.y, row.z, row.id)
        for row in geo.itertuples(index=False)
    }

    # Adjust input IDs to match geometry convention
    keys = zip((mid + 1 for mid in mpmtids), (sid + 1 for sid in posids))

    # Use the lookup dictionary to retrieve values efficiently
    results = [lookup.get(k, (-999.9, -999.9, -999.9, -999)) for k in keys]

    # Unpack results into separate arrays
    if len(results) == 0:
        return np.array([]), np.array([]), np.array([]), np.array([])
    x, y, z, c = map(np.array, zip(*results))

    return x, y, z, c


# Simple class to store a filtered time slice of hit data
class hit_slice:
    def __init__(self, hits):
        self.t = hits.t  # Hit times
        self.q = hits.q  # Hit charges
        self.x = hits.x  # x-positions
        self.y = hits.y  # y-positions
        self.z = hits.z  # z-positions
        self.cable = hits.cable 

# Class to process and store a collection of hits from an event
class hit_collection:
    def __init__(self, e, g=geo):
        # If 'e' is a DataFrame, extract directly the columns
        if isinstance(e, pd.DataFrame):
            df = e
            # Assume we already time constant filtered
            self.t       = df["hit_pmt_calibrated_times"].values
            self.q       = df["hit_pmt_charges"].values
            self.card    = df["hit_mpmt_card_ids"].values
            self.slot    = df["hit_mpmt_slot_ids"].values
            self.channel = df["hit_pmt_channel_ids"].values
            self.posid   = df["hit_pmt_position_ids"].values

            # Geometric Map
            self.x, self.y, self.z, self.cable = getxyz(g, self.slot, self.posid)

            # # Cut in geom
            # mask = self.x > -999
            # for attr in ("t", "q", "card", "slot", "channel", "posid", "x", "y", "z", "cable"):
            #     setattr(self, attr, getattr(self, attr)[mask])

            # Remove hits with invalid geometry mapping (sentinel x = -999.9)
            cut          = self.x > -999
            self.t       = self.t      [cut]
            self.q       = self.q      [cut]
            self.card    = self.card   [cut]
            self.slot    = self.slot   [cut]
            self.channel = self.channel[cut]
            self.posid   = self.posid  [cut]
            self.z       = self.z      [cut]
            self.y       = self.y      [cut]
            self.cable   = self.cable  [cut]
            self.x       = self.x      [cut]
            
        
        else:
            # Apply time window cut for hit calibration times
            above_10us  = np.array(e.hit_pmt_calibrated_times) > 10000
            below_490us = np.array(e.hit_pmt_calibrated_times) < 490000
            has_calib   = np.array(e.hit_pmt_has_time_constant)  # Check for valid calibration
            hit_selection = has_calib # Combined mask

            # Apply selection mask to extract relevant hit information
            self.t       = np.array(e.hit_pmt_calibrated_times) [hit_selection]
            self.q       = np.array(e.hit_pmt_charges)          [hit_selection]
            self.card    = np.array(e.hit_mpmt_card_ids)        [hit_selection]
            self.slot    = np.array(e.hit_mpmt_slot_ids)        [hit_selection]
            self.channel = np.array(e.hit_pmt_channel_ids)      [hit_selection]
            self.posid   = np.array(e.hit_pmt_position_ids)     [hit_selection]

            # Get spatial positions and cable IDs using geometry mapping
            self.x, self.y, self.z, self.cable = getxyz(g, self.slot, self.posid)

            # Remove hits with invalid geometry mapping (sentinel x = -999.9)
            cut          = self.x > -999
            self.t       = self.t      [cut]
            self.q       = self.q      [cut]
            self.card    = self.card   [cut]
            self.slot    = self.slot   [cut]
            self.channel = self.channel[cut]
            self.posid   = self.posid  [cut]
            self.z       = self.z      [cut]
            self.y       = self.y      [cut]
            self.cable   = self.cable  [cut]
            self.x       = self.x      [cut]

    # Return a new hit_slice object within a time window
    def time_slice(self, tstart, tend):
        h    = hit_slice(self)                                 # Create a new slice from this collection
        tsel = (h.cable > -1) & (h.t > tstart) & (h.t < tend)  # Select hits within the time range

        # Apply selection to all relevant hit attributes
        h.q     = h.q[ tsel ]
        h.x     = h.x[ tsel ]
        h.y     = h.y[ tsel ]
        h.z     = h.z[ tsel ]
        h.cable = h.cable[ tsel ]
        h.t     = h.t[ tsel ]      

        return h  # Return the filtered slice

In [3]:
# Load the geometry mapping into a DataFrame
runno = 1769
# ROOT Input
# tt    = get_offline_run_tchain(runno)

# DF Input
data   = pd.read_csv(f"../../data_analysis/NB/files/df_{runno}_forBONSAI_separated.csv", index_col=0)
events = np.unique(data["event_id"])
    
# Setup HKBONSAI with WCTE geo
simfile = ROOT.TFile("../bonsai_reqs/wcsim.root")
simtree = simfile.Get("wcsimGeoT")

Warning in <TClass::Init>: no dictionary for class pair<EBoundaryWallType,vector<float> > is available


In [6]:
geotree = None
for geoevent in simtree:
    geotree = geoevent.wcsimrootgeom
    break
    
bonsai = cppyy.gbl.WCSimBonsai()
bonsai.Init(geotree)

# Start Filter
ns = 1
count = 0
nhit_cut = 10
mvwindow_start =   10000*ns
mvwindow_end   = 5000000*ns
mvwindow_step  =      10*ns
mvwindow_width =      10*ns

vertex = {
"nhits": [],
"nhitso": [],
"x": [],
"y": [],
"z": [],
"result0": [],
"result1": [],
"result2": [],
"result3": [],
"result4": [],
"result5": [],
"good0":[],
"good1":[],
"good2":[] 
}

for df in tqdm(events, total=len(events)):
    count += 1    
    if count > 20000: break
    # print(count)

    # Generate hit_collection from event
    sub = data[data.event_id == df]
    hits_df = hit_collection(sub)

    # Run a moving average 50ns hit filter
    tstart = mvwindow_start 
    tend   = tstart + mvwindow_width
    try:
        tmax   = np.max(hits_df.t)
    except:
        print(df)
    
    while tend < tmax:
        
        window = hits_df.time_slice(tstart, tend)
        tstart += mvwindow_step
        tend   = tstart + mvwindow_width
    
        # Run prehit filter for this window
        if len(window.t) < nhit_cut: continue
        if len(window.t) > 400: continue
                
        # Run Bonsai
        bsVertex = array.array('f',3*[0.0])
        bsResult = array.array('f',6*[0.0])
        bsGood = array.array('f',3*[0.0])
        bsNhit = array.array('i',[len(window.cable)])
        bsNsel = array.array('i',[0])

        # Generate hit collection for this triggger
        bsCAB_a = array.array('i', window.cable)
        bsT_a = array.array('f', window.t - np.min(window.t) + 200)
        bsQ_a = array.array('f', window.q)

        # Run Bonsai
        try:
            nhits = bonsai.BonsaiFit(bsVertex, bsResult, bsGood, bsNsel, bsNhit, bsCAB_a, bsT_a, bsQ_a);
        except:
            print("BONSAIFAILED");
            pass
        # print(nhits, bsVertex, bsResult, bsGood, bsNsel, bsNhit)

        vertex["nhits"].append(nhits)
        vertex["nhitso"].append(len(window.t))
        
        vertex["x"].append(bsVertex[0])
        vertex["y"].append(bsVertex[1])
        vertex["z"].append(bsVertex[2])
        vertex["result0"].append(bsResult[0])
        vertex["result1"].append(bsResult[1])
        vertex["result2"].append(bsResult[2])
        vertex["result3"].append(bsResult[3])
        vertex["result4"].append(bsResult[4])
        vertex["result5"].append(bsResult[5])
        vertex["good0"].append(bsGood[0])
        vertex["good1"].append(bsGood[1])
        vertex["good2"].append(bsGood[2])
        
        # Skip to next window if we found a hit
        tstart += mvwindow_width
        tend    = tstart + mvwindow_width
        
for key in vertex:
    print(key, len(vertex[key]))
    
df = pd.DataFrame(vertex)
df.to_csv(f"df_test_vertex_{runno}.csv")

  5%|▌         | 20000/385435 [59:46<18:12:05,  5.58it/s]


nhits 4796
nhitso 4796
x 4796
y 4796
z 4796
result0 4796
result1 4796
result2 4796
result3 4796
result4 4796
result5 4796
good0 4796
good1 4796
good2 4796
 Geometry Loading (NPMT=1995) 
Loaded 1995 PMT locations, maximum r=157.713089, maximum z=138.848557
Loading pdfs from /mnt/netapp2/Store_uni/home/usc/ie/dcr/software/hk/hk-BONSAI/data/like.bin...
Turn on background fitting
pdf for -1.00pe<=q: zero at 730 total=2.086008ns
Double hit PMT  1917 error
Double hit PMT  1656 error
Double hit PMT   525 error
Double hit PMT  1715 error
Double hit PMT  1680 error
Double hit PMT   416 error
Double hit PMT  1586 error
Double hit PMT  1662 error
Double hit PMT   274 error
Double hit PMT    53 error
Double hit PMT   649 error
Double hit PMT  1651 error
Double hit PMT  1658 error
Double hit PMT  1040 error
Double hit PMT   689 error
Double hit PMT  1958 error
Double hit PMT   920 error
Double hit PMT  1032 error
